In [ ]:
pip install qiskit

In [ ]:
pip install qiskit_optimization

In [ ]:
pip install qiskit_ibm_runtime

In [ ]:
pip install qiskit_aer

In [ ]:
pip install qiskit_utils

In [ ]:
pip install dimod

In [ ]:
pip install dwave-ocean-sdk

In [ ]:
!pip install neal

ERROR: Could not find a version that satisfies the requirement neal (from versions: none)
ERROR: No matching distribution found for neal


In [ ]:
import numpy as np
import math

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, VQE, Grover
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session, Options
from qiskit_aer import AerSimulator, Aer
from typing import Dict, List, Tuple, Any, Optional
from qiskit_algorithms.utils import algorithm_globals
from qiskit_optimization.algorithms import GroverOptimizer
import dimod
from dimod import BQM
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave.samplers import SimulatedAnnealingSampler

In [ ]:
class Optimizer:

    REQUIRED_PARAMS = []

    def __init__(self, params) -> None:

        for label in self.REQUIRED_PARAMS:  # make sure we have all the required params
            if label not in params:
                raise ValueError(f"Please provide {label} in params.")

        self.params = params.copy()

        # self.results: Dict[str, DistributedEnergyOptimizerResults] = {}
        self._quadratic_program: Optional[QuadraticProgram] = None
        self._linear_terms: Optional[Dict[str, float]] = None
        self._quadratic_terms: Optional[Dict[Tuple[str, str], float]] = None
        self._offset: Optional[float] = None

1. Linear and Quadratic Coeffs


2. Creating the QUBO


In [ ]:
def quadratic_prog_creation(self) -> QuadraticProgram:

  qp = QuadraticProgram(name="path")
  binary_vars = []

  # add binary vars to qubo
  for i in binary_vars:
    qp.binary_var(name=i)

  qp.minimize(linear=self.linear_terms,quadratic=self.quadratic_terms,constant=self.offset) # TODO: offset

  return qp

3. Solve QUBO

In [ ]:
def solve_qubo(self, opt_type) -> str:

  # set simulator
  # simulator = AerSimulator()
  # simulator.set_options(seed_simulator=algorithm_globals.random_seed, seed_transpiler=algorithm_globals.random_seed)

  # define algorithm
  optimizers = {"qaoa": QAOA, "vqe": VQE}
  chosen_optimizer = optimizers[opt_type]
  algorithm = chosen_optimizer(sampler=Sampler(), optimizer=COBYLA())

  # solve
  optimizer=MinimumEigenOptimizer(algorithm)
  result = optimizer.solve(self.quadratic_program)

In [ ]:
def run_grover(self):

  # set simulator
  simulator = AerSimulator()
  # simulator.set_options(seed_simulator=algorithm_globals.random_seed, seed_transpiler=algorithm_globals.random_seed)

  n = self.params["n"]
  N = self.params["N"]
  num_qubits = n + N * n

  grover_optimizer = GroverOptimizer(num_qubits, num_iterations=10, sampler=simulator)
  results = grover_optimizer.solve(self.quadratic_program)
  print(results.prettyprint())

In [ ]:
#fake test!

linear_coefficients = {}
quadratic_coefficients = {}
qp = QuadraticProgram()
binary_vars = []

weights_matrix = np.zeros((2,2))
for i in range(2):
  for j in range(2):
    weights_matrix[i][j] = i*j + 1

for i in range(2):
    for j in range(2):
        var = qp.binary_var(name=f'x_{i}_{j}')
        binary_vars.append(var)
        linear_coefficients[f'x_{i}_{j}'] = weights_matrix[i, j]
        if j > i:
            quadratic_coefficients[(f'x_{i}_{j-1}', f'x_{i+1}_{j}')] = weights_matrix[i, j]

In [ ]:
def run_bqm():

vartype = 'BINARY'

# define BQM
linear_terms, quadratic_terms = linear_coefficients, quadratic_coefficients

bqm = dimod.BinaryQuadraticModel(
    linear_terms, quadratic_terms, vartype=vartype
)

sampler = SimulatedAnnealingSampler()
response = sampler.sample(bqm, num_reads=100)

# Print the best solution
print("Best solution:", response.first.sample)

# Print the energy of the best solution
print("Num of the best solution:", response.first.energy)

# Print the number of occurrences of the best solution
print("Number of occurrences of the best solution:", response.first.num_occurrences)

Best solution: {'x_0_0': 0, 'x_0_1': 0, 'x_1_0': 0, 'x_1_1': 0}
Num of the best solution: 0.0
Number of occurrences of the best solution: 1


In [ ]:
def run_qaoa(self):
  solve_qubo(self, "qaoa")

def run_qaoa(self):
  solve_qubo(self, "vqe")